In [434]:
import pandas as pd

In [435]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [436]:
df = df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns')
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [437]:
target = df.Survived
inputs = df.drop(['Survived'], axis='columns')
inputs.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [438]:
dummies = pd.get_dummies(inputs.Sex).astype(int)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [439]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [440]:
inputs = inputs.drop('Sex',axis='columns')
inputs.head(3)

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0


In [441]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [442]:
inputs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Age     714 non-null    float64
 2   Fare    891 non-null    float64
 3   female  891 non-null    int32  
 4   male    891 non-null    int32  
dtypes: float64(2), int32(2), int64(1)
memory usage: 28.0 KB


In [443]:
inputs.Age = inputs.Age.fillna(inputs.Age.median())
print(inputs.info())
print(inputs.columns[inputs.isna().any()])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Age     891 non-null    float64
 2   Fare    891 non-null    float64
 3   female  891 non-null    int32  
 4   male    891 non-null    int32  
dtypes: float64(2), int32(2), int64(1)
memory usage: 28.0 KB
None
Index([], dtype='object')


In [444]:
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [445]:
from sklearn.model_selection import train_test_split

In [446]:
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2)
len(inputs),len(X_train),len(X_test)

(891, 712, 179)

In [447]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB() 

In [448]:
model.fit(X_train,y_train)

GaussianNB()

In [449]:
model.score(X_test,y_test)

0.8044692737430168

In [450]:
X_test[0:10]

,Pclass,Age,Fare,female,male
824,3,2.0,39.6875,0,1
384,3,28.0,7.8958,0,1
856,1,45.0,164.8667,1,0
484,1,25.0,91.0792,0,1
761,3,41.0,7.1250,0,1
381,3,1.0,15.7417,1,0
271,3,25.0,0.0000,0,1
639,3,28.0,16.1000,0,1
174,1,56.0,30.6958,0,1
697,3,28.0,7.7333,1,0


In [451]:
y_test[0:10]

824    0
384    0
856    1
484    1
761    0
381    1
271    1
639    0
174    0
697    1
Name: Survived, dtype: int64

In [452]:
model.predict(X_test[0:10])

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 1], dtype=int64)

In [453]:
predictions = pd.Series(model.predict(X_test[0:10]), name="Predicted")
actual = y_test[0:10].reset_index(drop=True) 
h = pd.concat([predictions, actual], axis='columns')
h

,Predicted,Survived
0,0,0
1,0,0
2,1,1
3,0,1
4,0,0
5,1,1
6,0,1
7,0,0
8,0,0
9,1,1


In [454]:
model.predict_proba(X_test[:10])

array([[9.69721314e-01, 3.02786861e-02],
       [9.88702429e-01, 1.12975710e-02],
       [6.89829225e-06, 9.99993102e-01],
       [5.68190892e-01, 4.31809108e-01],
       [9.88942704e-01, 1.10572959e-02],
       [4.17474990e-02, 9.58252501e-01],
       [9.87828192e-01, 1.21718083e-02],
       [9.88663128e-01, 1.13368719e-02],
       [8.74602673e-01, 1.25397327e-01],
       [8.89781995e-02, 9.11021800e-01]])